# Calculate confounders and combine with those from GEE

In [ ]:
#load libraries
library(tidyverse)
library(sf)
library(raster)

In [ ]:
##set path to datasets
data_path = "PATH"
##set the temp directory for raster processing 
rasterOptions(tmpdir="~/scratch/")

In [ ]:
#load in the reference raster (all zeros) 
ref <- raster(paste(data_path,'zam_ref_0.tif', sep = "/"))

##convert to meter based coordinate system
ref_rp <- projectRaster(ref, crs='+proj=merc +lon_0=0 +k=1 +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs')

##set up reference with cell ID values
ref_m <- setValues(raster(ref_rp),seq(1:length(ref_rp))) + ref_rp
names(ref_m) <- 'ID'

In [ ]:
##save the refrence raster
#writeRaster(ref_m, paste(data_path, "zam_ref_m.tif",  sep = "/"), overwrite=TRUE)

In [ ]:
#read in vector zam outline (taken form GEE to match those confounders)
zam <- st_read(dsn = paste(data_path,'zam_outline', sep = "/")  , layer="zam_outline") %>%
            st_transform(st_crs(ref_m))

In [ ]:
## set number of years
years <- seq(2000,2019)

In [ ]:
##read in the population density datasets
pop <- list.files(path = paste0(data_path,'R_conf/pop_density/'),pattern = "*.tif")

##loop through files read in and add to a raster stack
p_stack <- stack(ref_m)

for (i in 1:length(pop)) {
    p  <- raster(paste0(data_path,'R_conf/pop_density/',pop[i]))
    p_m <- projectRaster(p, ref_m)
    p_stack <- stack(p_stack,p_m)
                               }

In [ ]:
##set names to years and ID
names(p_stack) <- c('ID',years)

In [ ]:
##convert to a dataframe and pivot to long format
pop_df <- as.data.frame(p_stack) %>%
            pivot_longer(cols = starts_with('X'), names_to = 'year', values_to = 'pop_den') %>%
            filter(ID >0) %>% 
            mutate(pop_den = replace_na(pop_den,0), year = parse_number(year))

In [ ]:
##read in vmap0 dataset of roads reproject to meters system
roads_97 <- st_read(dsn = paste(data_path,'R_conf/soa_vmap0', sep = "/")  , layer="trans-road-l")  %>%
            st_transform(st_crs(ref_m))

In [ ]:
##clip roads to zam outline
zam_r_97 <- roads_97[zam, ]
length(zam_r_97)

In [ ]:
##rasterize the roads to reference
roads.97.r <- rasterize(zam_r_97, ref_m, field=1)

In [ ]:
##calculate distance to a road cell - this is in meters
road_dist_97 <- gridDistance(roads.97.r, origin = 1)

In [ ]:
##read in gRoads dataset and reproject to meters system
roads_10 <- st_read(dsn = paste(data_path,'R_conf/groads-v1-africa-shp', sep = "/"), layer="gROADS-v1-africa")  %>%
            st_transform(st_crs(ref_m))

In [ ]:
##clip roads to zam outline
zam_r_10 <- roads_10[zam, ]
length(zam_r_10)

In [ ]:
##rasterize the roads to the reference
roads.10.r <- rasterize(zam_r_10, ref_m, field=1)

In [ ]:
##calcaute distance on the road raster - in meters
road_dist_10 <- gridDistance(roads.10.r, origin = 1)

In [ ]:
##create a stack with layer per year (values are the same 2000-2009 and 2010-2019) 
road_stack <- stack(ref_m)

for (i in 1:length(years)) {
    if (1 +1999 < 2010){
        road_stack <- stack(road_stack,road_dist_97)
    }else{
        road_stack <- stack(urb_stack,road_dist_10)
}}

In [ ]:
#set names as years and cell ID
names(road_stack) <- c('ID',years)

In [ ]:
##convert to a dataframe and pivot to long format
road_df <- as.data.frame(road_stack) %>%
            pivot_longer(cols = starts_with('X'), names_to = 'year', values_to = 'road_dist') %>%
            filter(ID >0) %>% 
            mutate(year = parse_number(year))

In [ ]:
##read in Zambian provinces
adm_1 <- st_read(dsn = paste(data_path,'zam_outline', sep = "/"), layer = 'gadm36_ZMB_1') %>%
            st_transform(st_crs(ref_m))

In [ ]:
##merge central and luska provience (some mines cross the coundry and luska is very small)
adm_1_m <-  adm_1 %>% 
            filter(NAME_1 %in% c('Lusaka', 'Central')) %>% 
            st_union() %>% 
            st_sf() %>% # make the geometry a data frame object
            mutate(NAME_1 = 'Lusaka_Central') %>% 
            rbind(filter(adm_1, NAME_1 %in% c('Copperbelt','Eastern','Luapula','Muchinga',
                                              'North-Western', 'Northern','Southern','Western'))[,c('geometry', 'NAME_1')]) %>%
            st_buffer(dist = 10000)

In [ ]:
##convert to a raster
adm_1_r <- rasterize(adm_1_m, ref_m, field = as.factor(adm_1_m$NAME_1))
names(adm_1_r) <- 'province'

In [ ]:
##read in Agroecological zone dataset 
agri_zone_in <- raster(paste(data_path,"R_conf/cell5m_Agroecology_Agro_EcologicalZone_AEZ5_CLAS.tif",sep="/"))

In [ ]:
##resample to referenece - with nearest value taken
agri_zone <- projectRaster(agri_zone_in, ref_m, method="ngb")
names(agri_zone) <- 'agri_zone'

In [ ]:
## read in the counfoudners from Google Earth Engine
gee_conf <- brick(paste0(data_path,'GEE_conf.tif'))

In [ ]:
## project to the meters based system and stack with the referenece raster 
gee_conf <- projectRaster(gee_conf, ref_m)
gee_stack <- stack(ref_m, gee_conf)

In [ ]:
#convert to dataframe and pivot to long form
gee_df <- as.data.frame(gee_stack) %>%
            pivot_longer(cols = starts_with('X'), names_to = 'year', values_to = 'burned') %>%
            mutate(burned = replace_na(burned,0), year = parse_number(year)) %>%
            filter(ID > 0)

In [ ]:
## calculate burning in the previous two years
gee_df <- gee_df %>%
            group_by(ID) %>%
            arrange(year, .by_group = TRUE) %>%
            mutate(burn_pre_yr = lag(burned, default = first(burned)),
                   burn_pre_2yr = burn_pre_yr + lag(burn_pre_yr, defualt = first(burn_pre_yr))) %>%
            ungroup()

In [ ]:
##load in the Hansen forest loss datasets created in GEE
tc = brick(paste0(data_path,'hansen/tc_1km.tif')) # pertage tree cover masked to only 0.5 ha forest pixels
loss = brick(paste0(data_path,'hansen/la_fmasked_1km.tif')) ##area lost (sum of 30m pixels unweighted be tree cover) masked to remove non-forest in 2000 
loss_tcw = brick(paste0(data_path,'hansen/la_fm_w_1km.tif')) ## area lost masked with forest at 0.5 ha and weighted by tree cover
tc_sd = brick(paste0(data_path,'hansen/tc_1km_sd.tif')) ## standard devation of tc at 1km 

In [ ]:
##set names as years
names(tc) = years
names(loss) = years 
names(loss_tcw) = years
names(tc_sd) = years

In [ ]:
##reproject to the meters based system
tc = projectRaster(tc, crs=crs(ref_m))
loss = projectRaster(loss, crs=crs(ref_m))
loss_tcw = projectRaster(loss_tcw, crs=crs(ref_m))
tc_sd = projectRaster(tc_sd, crs=crs(ref_m))

In [ ]:
##convert to dataframes with the ID value and pivot to long form 
tc_long =   as.data.frame(stack(ref_m,tc)) %>% 
            pivot_longer(cols = starts_with('X'), names_to = 'year', values_to = 'tc', names_prefix= 'X') %>% 
            drop_na()

loss_long = as.data.frame(stack(ref_m,loss)) %>% 
            pivot_longer(cols = starts_with('X'), names_to = 'year', values_to = 'loss', names_prefix= 'X') %>% 
            drop_na()

loss_tcw_long = as.data.frame(stack(ref_m,loss_tcw)) %>% 
                pivot_longer(cols = starts_with('X'), names_to = 'year', values_to = 'loss_tcw', names_prefix= 'X') %>% 
                drop_na()

tc_sd_long =   as.data.frame(stack(ref_m,tc_sd)) %>% 
                pivot_longer(cols = starts_with('X'), names_to = 'year', values_to = 'tc_sd', names_prefix= 'X') %>% 
                drop_na()

In [ ]:
##create a single forest cover and loss dataframe
trees <- left_join(tc_long, loss_long, by =c('ID','year')) %>% 
                    left_join(loss_tcw_long, by =c('ID','year')) %>% 
                        left_join(tc_sd_long, by = c('ID', 'year'))

In [ ]:
##create tc in the previous year and loss in the previous two years - values needed for matching
trees <- trees %>%
            group_by(ID) %>%
            arrange(year, .by_group = TRUE) %>%
            mutate(year = as.numeric(year),
                   tc_pre_yr = lag(tc, default = first(tc)), 
                   loss_pre_yr = lag(loss, default = first(loss)),
                   loss_pre_2yr = loss_pre_yr + lag(loss_pre_yr, default = first(loss_pre_yr)),
                   loss_2yr_bin = ifelse(loss_pre_2yr >0,1,0)) %>%
            ungroup()

In [ ]:
##combine the confounders that don't vary in time
fixed <- stack(ref_m, adm_1_r, agri_zone)

In [ ]:
##convert to a dataframe
fixed_df <- as.data.frame(fixed, xy = TRUE) %>%
            filter(ID >0)

In [ ]:
##combie all confounders into one dataset
conf <- pop_df %>%
        left_join(road_df, by = c('ID', 'year')) %>%
        left_join(fixed_df, by = 'ID') %>%
        left_join(gee_df, by = c('ID', 'year')) %>%
        left_join(trees, by = c('ID', 'year'))

In [ ]:
length(unique(conf$ID))
length((conf$ID))

In [ ]:
##save as a csv
#write.csv(conf,paste0(data_path,'dfs/confounders.csv'), row.names = FALSE)